# Калькулятор

Реализуйте генератор `calculator`, который выполняет арифметические операции на переданных ему данных. Генератор в качестве данных получает кортеж из операции и числа, отправляет обратно результат выполнения операции.

Генератор должен поддерживать следующие команды через метод `send`:

- `add` — прибавить переданное значение к текущему.
- `subtract` — вычесть переданное значение.
- `multiply` — умножить текущее значение на переданное.
- `divide` — разделить текущее значение на переданное.

Генератор должен принимать начальное значение при инициализации, и последовательно модифицировать его через команды, отправленные с помощью `send`.

В случае закрытия генератор должен возвращать None и переставать генерировать значения.

В случае отправки исключений в генератор они должны заглушаться. (Не заглушаем `ZeroDivisionError`, `ValueError` и `GeneratorExit`)

In [ ]:
from typing import Generator
import unittest
from enum import Enum


class Operation(Enum):
    ADD = "add"
    MULTIPLY = "multiply"
    SUBTRACT = "subtract"
    DIVIDE = "divide"
    UNKNOWN = "unknown"


def calculator(
    start_value: float = 0.0,
) -> Generator[float, tuple[Operation, float], None]: ...


class TestCalculator(unittest.TestCase):
    def setUp(self):
        """Инициализация калькулятора перед каждым тестом"""
        self.calc = calculator(10)
        next(self.calc)

    def test_calculator_basic_operations(self):
        """Тест базовых операций калькулятора"""
        self.assertEqual(self.calc.send((Operation.ADD, 5)), 15)
        self.assertEqual(self.calc.send((Operation.MULTIPLY, 3)), 45)
        self.assertEqual(self.calc.send((Operation.SUBTRACT, 15)), 30)
        self.assertEqual(self.calc.send((Operation.DIVIDE, 2)), 15)

    def test_calculator_zero_division(self):
        """Тест деления на ноль"""
        with self.assertRaises(ZeroDivisionError) as context:
            self.calc.send((Operation.DIVIDE, 0))
        self.assertEqual(str(context.exception), "Cannot divide by zero")

    def test_calculator_unknown_operation(self):
        """Тест неизвестной операции"""
        with self.assertRaises(ValueError) as context:
            self.calc.send((Operation.UNKNOWN, 5))
        self.assertEqual(str(context.exception), "Unknown operation: unknown")

    def test_calculator_close(self):
        """Тест закрытия калькулятора"""
        self.calc.close()
        with self.assertRaises(StopIteration):
            next(self.calc)

    def test_calculator_throw(self):
        """Тест принудительного исключения"""
        self.calc.throw(AttributeError("Manual throw"))


if __name__ == "__main__":
    unittest.TextTestRunner(verbosity=2).run(
        unittest.TestLoader().loadTestsFromTestCase(testCaseClass=TestCalculator)
    )

# Бесконечный генератор

Нужно реализовать функцию, принимающую итерируемый объект и возвращающая генератор, который бесконечно повторяет все поэлементно входной объект. Пример для понимания:

```python
g = inf_gen([1, 2])

assert next(inf_gen) == 1
assert next(inf_gen) == 2
assert next(inf_gen) == 1
assert next(inf_gen) == 2
assert next(inf_gen) == 1

```

Необходимо пробрасывать `ValueError` если в функцию передан неитерируемый объект!

In [ ]:
from typing import Iterable, Any
from itertools import islice


def inf_gen(iterable: Iterable[Any]) -> Iterable[Any]: ...


class TestInfiniteGenerator(unittest.TestCase):
    def test_sequence(self):
        generator = inf_gen(iterable="abc")
        assert list(islice(generator, 6)) == ["a", "b", "c", "a", "b", "c"]

    def test_empty(self):
        generator = inf_gen(iterable=())
        assert list(generator) == []

    def test_non_iterable(self):
        with self.assertRaises(ValueError):
            generator = inf_gen(iterable=5)
            next(generator)

    def test_iterator(self):
        generator = inf_gen(iterable=iter("abc"))
        assert list(islice(generator, 6)) == ["a", "b", "c", "a", "b", "c"]

    def test_generator(self):
        generator = inf_gen(iterable=(i for i in range(3)))
        assert list(islice(generator, 6)) == [0, 1, 2, 0, 1, 2]

    def test_inf_generator(self):
        generator = inf_gen(iterable=inf_gen([1]))
        assert list(islice(generator, 6)) == [1, 1, 1, 1, 1, 1]


if __name__ == "__main__":
    unittest.TextTestRunner(verbosity=2).run(
        unittest.TestLoader().loadTestsFromTestCase(testCaseClass=TestInfiniteGenerator)
    )

# Разворачивающий

Реализуйте класс `Reversed`.

`Reversed` - итератор, который итерируется по значениям из входного объекта с конца. 

Класс `Reversed` должен полностью реализовывать интерфейс последовательности.

Если передается объект большой размерности, то обрезайте его до 1000 первых элементов.

Разрешается исчерпать входной итерируемый объект.

In [ ]:
from collections.abc import Sequence
from typing import Iterable, Any
from itertools import count


class Reversed[Any](Sequence): ...


class TestReversed(unittest.TestCase):
    def test_sequence(self):
        r = Reversed(iterable="abc")
        assert list(islice(r, 3)) == ["c", "b", "a"]

    def test_raises_stop_iteration(self):
        r = Reversed(iterable="abc")
        assert list(islice(r, 3)) == ["c", "b", "a"]
        with self.assertRaises(StopIteration):
            next(r)

    def test_empty(self):
        r = Reversed(iterable=())
        assert list(islice(r, 3)) == []

    def test_iterator(self):
        r = Reversed(iterable=iter([1, 2, 3]))
        assert list(islice(r, 2)) == [3, 2]

    def test_generator(self):
        r = Reversed(iterable=(_ for _ in range(10001)))
        assert list(islice(r, 4)) == [999, 998, 997, 996]

    def test_limited(self):
        r = Reversed(iterable=iter([1, 2, 3]), max_size=1)
        assert list(islice(r, 2)) == [1]

    def test_inf_generator(self):
        r = Reversed(iterable=count(0, 1))
        assert list(islice(r, 4)) == [999, 998, 997, 996]

    def test_len(self):
        r = Reversed(iterable=count(0, 1))
        assert len(r) == 1000
        assert list(islice(r, 4)) == [999, 998, 997, 996]

    def test_getitem(self):
        r = Reversed(iterable=count(0, 1))
        assert r[-1] == 0
        assert list(islice(r, 4)) == [999, 998, 997, 996]

    def test_next(self):
        r = Reversed(iterable=count(0, 1))
        assert next(r) == 999
        assert next(r) == 998

    def test_reversed(self):
        r = Reversed(Reversed(iterable="abc"))
        assert list(islice(r, 3)) == ["a", "b", "c"]


if __name__ == "__main__":
    unittest.TextTestRunner(verbosity=2).run(
        unittest.TestLoader().loadTestsFromTestCase(testCaseClass=TestReversed)
    )